# Python 时间概念与常用库

现实中的实践都是在空间与时间中进行的，在多个时间点观测或采集的数据就构成时间序列数据。在科学、经济、金融等领域，时间序列数据是一种重要的结构化数据形式，应用广泛。

在现实中最常用的是时刻时间，例如9点钟开会，9点就是一个时刻概念，是时间轴的一个点。还会用到时间段概念，例如9点开始开半个小时的会，意思是9点到9点30分中的时间段。还有一个常用的时间概念是时间差，例如9点差5分，9点过5分，这里的 `-5`，`+5` 就是时间差。在 Python 中也有多个时间处理模块，本节介绍时间的基本概念，常用库，以及常用操作。

In [22]:
import time
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity='all'

## Python 时间类型

众所周知地球是圆的，不同地方并不同时日出。目前大部分国家均使用协调世界时（Coordinated Universal Time, UTC），各个地方则通过时区差（与UTC时间的差）形成地方时，使用地方时既符合当地人的日常作息，也保证时间的统一。格林尼治天文台位于伦敦的一个区，其时区差为0，可以说格林尼治时间就是 UTC 时间。而中国时区为东8区，比格林尼治时间快8个小时，即C+8。

计算机的时间用用整数或浮点数来表示，是从1970年1月1日零时零分零秒开始计时的秒数，称为 Unix 时间戳。Unix 时间戳是协调世界时（UTC）。Pyhton 的 Unix 时间戳用浮点数表示，可以使用`time`模块函数`time.time()`可以获得当前时间（浮点数）：

In [2]:
t0 = time.time()
print(type(t0), t0)

<class 'float'> 1556075534.0913515


`datetime`模块提供有日期数据类型（`datetime.datetime`类），比浮点数时间的功能更多，使用也方便。调用类方法可以获得日期类型时间：

In [3]:
now = datetime.datetime.utcnow()
print(type(now), now)

<class 'datetime.datetime'> 2019-04-24 03:12:14.123264


`datetime`模块还提供了时间差的数据类型（`datetime.timedelta`类），用来表示两个日期对象之间的差：

In [4]:
deltat = datetime.datetime.now() - datetime.datetime(2008, 10, 1)
print(type(deltat), deltat)
# 时间差：多少天，多少秒，或者总共是多少秒
print(deltat.days, deltat.seconds, deltat.total_seconds())

<class 'datetime.timedelta'> 3857 days, 11:12:14.138225
3857 40334 333285134.138225


大部分场景，使用便于人类阅读的字符串来表示日期时间，不同国家具有不同的日期阅读习惯。例如：

In [5]:
# 北京奥运会开幕式时间
s = '2018-08-08 20:00:00'
s

'2018-08-08 20:00:00'

**不同时间类型的转换**

`datetime.datetime`类提供了两个方法实现Unix时间戳与 `datetime` 对象的相互转换：

In [6]:
# 转换Unix时间戳为 `datetime`时间
datetime.datetime.fromtimestamp(t0)

datetime.datetime(2019, 4, 24, 11, 12, 14, 91352)

In [7]:
# 转换`datetime` 时间为Unix时间戳
now.timestamp()

1556046734.123264

`datetime` 日期对象提供多种方法转换为字符串：

In [8]:
# 使用`isoformat()`转换为ISO标准格式
print(now.isoformat())
# 使用`strftime()` 方法自定义
print(now.strftime('%Y-%m-%d %H:%M:%S'))

2019-04-24T03:12:14.123264
2019-04-24 03:12:14


`datetime.datetime`类提供方法来解析字符串：

In [9]:
s = '2018-08-08 20:00:00'
datetime.datetime.strptime(s, '%Y-%m-%d %H:%M:%S')

datetime.datetime(2018, 8, 8, 20, 0)

使用 `dateutil` 库可以解析大部分人类可理解的日期表示形式：

In [10]:
from dateutil import parser

s = '2018-08-08 20:00:00'
opening_date2 = parser.parse(s)
opening_date2

datetime.datetime(2018, 8, 8, 20, 0)

在有些国家，日期出现在月份之前，通过传入参数`dayfirst=True`来解析这类日期字符串：

In [11]:
s = '31/12/2018 11:00 PM'
parser.parse(s, dayfirst=True)

datetime.datetime(2018, 12, 31, 23, 0)

## Numpy 的时间类型

无论是 Unix 时间戳，`datetime.datetime` 时间，亦或 `datetime.timedelta` 时间差，其时间分辨率都是微妙量级。在 Numpy 中提供了分辨率达到纳秒的时间对象：
- 时间戳对象 `datetime64`
- 时间差对象 `timedelta64`

下面创建 Numpy 的时间戳与时间差对象：

In [12]:
xdt64 = np.datetime64('2018-08-08 20:00:00')
ydt64 = np.datetime64(now)
type(xdt64), xdt64, ydt64

(numpy.datetime64,
 numpy.datetime64('2018-08-08T20:00:00'),
 numpy.datetime64('2019-04-24T03:12:14.123264'))

In [13]:
xtd64 = np.timedelta64(86400, 's')
ytd64 = np.timedelta64(deltat)
type(xtd64), xtd64, ytd64

(numpy.timedelta64,
 numpy.timedelta64(86400,'s'),
 numpy.timedelta64(333285134138225,'us'))

In [14]:
str(xdt64)

'2018-08-08T20:00:00'

## Pandas的时间类型

Pandas 初心之一就是要集成时间序列功能，提供一组标准的时间序列处理工具和数据算法。在 Pandas 中提供了如下时间有关类型：
- 时间戳（`pd.Timestamp`），特定的时刻；
- 时间差（`pd.Timedelta`），时间戳之差；
- 时间间隔（`pd.Interval`），由起始时间和结束时间戳表示；
- 固定时期（`pd.Period`），表示一个标准的时间段(例如某年、某日等)，时间长短由频率决定。时期可以认为是时间间隔的特殊情况。

### Pandas 时间戳

可以使用`pd.Timestamp`类或`pd.to_datetime()` 函数来创建 Pandas 时间戳对象：

In [15]:
pt0 = pd.Timestamp(now)
pt1 = pd.to_datetime('2018-08-08 20:00:00')
type(pt0), pt0, pt1

(pandas._libs.tslibs.timestamps.Timestamp,
 Timestamp('2019-04-24 03:12:14.123264'),
 Timestamp('2018-08-08 20:00:00'))

使用`to_pydatetime()`转换为`datetime.datetime`对象：

In [16]:
pt0.to_pydatetime()

datetime.datetime(2019, 4, 24, 3, 12, 14, 123264)

使用`strftime()`等方法来转换为字符串：

In [17]:
# iso 格式
print(pt0.isoformat())
print(pt0.strftime('%Y-%m-%d %H:%M:%S'))

2019-04-24T03:12:14.123264
2019-04-24 03:12:14


Pandas 主要还是用来处理数组的，使用`pd.to_datetime()`函数传入类数组对象，可以转换字符串，同时还能处理缺失值：

In [18]:
idx = pd.to_datetime(['2018-08-08 20:00:00', None, ''])
idx

DatetimeIndex(['2018-08-08 20:00:00', 'NaT', 'NaT'], dtype='datetime64[ns]', freq=None)

可以看出，返回一个`DatetimeIndex`对象，使用 Numpy 的 `datetime64` 数据类型来存储时间戳，分辨率达到纳秒。`pd.NaT` （Not a Time）是 Pandas 时间戳数据的缺失值。

`DatetimeIndex`对象中的标量数据是 Pandas 时间戳：

In [19]:
type(idx[0]), idx[0]

(pandas._libs.tslibs.timestamps.Timestamp, Timestamp('2018-08-08 20:00:00'))

### Pandas 时间差

Pandas 时间差对象用来表示时间戳之差，可以使用`pd.Timedelta`类或使用时间戳之差来获得：

In [20]:
dt0 = pd.Timedelta(days=1, seconds=10)
dt1 = pt0 - pt1
type(dt0), dt0, dt1

(pandas._libs.tslibs.timedeltas.Timedelta,
 Timedelta('1 days 00:00:10'),
 Timedelta('258 days 07:12:14.123264'))

使用`total_seconds()`返回时间差总秒数：

In [21]:
dt.total_seconds()

NameError: name 'dt' is not defined

### Pandas 时间间隔（interval）

Pandas 时间间隔（interval）对象用来表示起始时间和结束时间戳之间的时间段。使用`pd.Interval`类来创建对象：

In [ ]:
year_2018 = pd.Interval(pd.Timestamp('2018-01-01 00:00:00'),
                        pd.Timestamp('2019-01-01 00:00:00'),
                        closed='left')
year_2018

可以使用 `in` 运算符来检查指定时间戳是否在时间间隔内：

In [ ]:
print(pd.Timestamp('2018-01-01 00:00') in year_2018)
print(pd.Timestamp('2018-06-01 00:00') in year_2018)

其属性`length`返回结束与起始时间戳的时间差：

In [ ]:
year_2018.length

### Pandas 时间区间

Pandas 时间区间表示一个固定的时间间隔。使用`pd.Period()`来创建对象：

In [ ]:
pp0 = pd.Period('2018-10-01', 'D')
type(pp0), pp0

## 创建时间数组

Pandas 的时间序列数据分为等间隔与不等间隔。当时序不规则，即数据之间的时间间隔不相等，称为不等间隔时间序列。不过很多时候需要处理等间隔的数据，也即需要处理固定频率的数据。例如在股票中，需要处理每分钟，每小时，每天的频率的数据。Pandas 支持多种时间序列频率，可用字符串或`pd.DateOffset`偏置对象来指定，下表仅列出几个常用的时间序列频率：

| 别名  | 偏置类型 |  描述    | 别名  | 偏置类型 |  描述   |
|:------|:---------|:-----------|:-----|:---------|:------|
| D    | Day    | 每天（日历）| B   | BusinessDay | 每天（工作） |
| H    | Hour   | 每小时    | T/min  | Minute | 每分钟   |
| S    | Second  | 每秒     | L/ms  | Milli | 每毫秒 |
| M    | MonthEnd | 每月月底日期（日历）   | BM   | BusinessMonthEnd | 每月月底日期（工作） |
| W-MON等 | Week | 每周（周一）等 | Day    | A-JAN   | BusinessYearBegin | 每年（1月） |

可以从模块`pd.tseries.offsets`获得常用偏置类型：

In [37]:
print([attr for attr in dir(pd.tseries.offsets) if attr[0].isupper()])

['ABCDatetimeIndex', 'ABCPeriod', 'ABCSeries', 'ApplyTypeError', 'BDay', 'BMonthBegin', 'BMonthEnd', 'BQuarterBegin', 'BQuarterEnd', 'BYearBegin', 'BYearEnd', 'BaseOffset', 'BusinessDay', 'BusinessHour', 'BusinessHourMixin', 'BusinessMixin', 'BusinessMonthBegin', 'BusinessMonthEnd', 'CBMonthBegin', 'CBMonthEnd', 'CDay', 'CustomBusinessDay', 'CustomBusinessHour', 'CustomBusinessMonthBegin', 'CustomBusinessMonthEnd', 'DateOffset', 'Day', 'Easter', 'FY5253', 'FY5253Quarter', 'Hour', 'LastWeekOfMonth', 'Micro', 'Milli', 'Minute', 'MonthBegin', 'MonthEnd', 'MonthOffset', 'Nano', 'OutOfBoundsDatetime', 'QuarterBegin', 'QuarterEnd', 'QuarterOffset', 'Second', 'SemiMonthBegin', 'SemiMonthEnd', 'SemiMonthOffset', 'SingleConstructorOffset', 'Tick', 'Timedelta', 'Timestamp', 'Week', 'WeekOfMonth', 'YearBegin', 'YearEnd', 'YearOffset']


Pandas 中时间戳、时间差等对象通常会构成相应的索引对象或数组，如：
- `pd.DatetimeIndex`，时间戳索引对象。
- `pd.TimedeltaIndex`，时间差索引对象。
- `pd.IntervalIndex`，时间间隔索引对象。
- `pd.PeriodIndex`，时间区间索引对象。

### 创建 `DatetimeIndex` 

可以使用函数`pd.to_datetime()`，`pd.date_range()`以及`pd.DatetimeIndex`类来创建时间戳索引对象。`pd.date_range()`函数根据指定频率生成指定长度的时间戳索引对象，其使用语法为：
```python
pd.date_range(start=None, end=None, periods=None, freq=None, tz=None, normalize=False, name=None, closed=None, **kwargs)
```
主要参数：
- `start, end`，起始时间与结束时间
- `periods`，周期数目，可与起始时间或结束时间配合，创建指定长度时间戳数据
- `freq`，时间序列频率
- `tz`，指定时间数据的时区
- `closed=None`，指定区间的开闭状态

下面使用起始时间、结束时间与长度来创建时间戳索引：

In [ ]:
# 指定起始与结束时间
pd.date_range(start='2018-01-01', end='2018-01-31')

In [ ]:
# 指定起始时间，与数据长度，
pd.date_range(start='2018-01-01', periods=10)

缺省情况，使用时间频率为日。下面来创建工作日日期：

In [ ]:
pd.date_range(start='2018-01-01', periods=10, freq='B')

### 创建`TimedeltaIndex`

除了使用`pd.TimedeltaIndex`类创建索引对象外，Pandas 提供`pd.timedelta_range`来创建`TimedeltaIndex`对象：

In [ ]:
# 指定时间差
pd.timedelta_range(start='1 day', periods=4, closed='left')

### 创建 `IntervalIndex`

使用`pd.interval_range()`函数来创建`pd.IntervalIndex`对象：

In [ ]:
# 指定起始与结束时间
pd.interval_range(start=pd.Timestamp('2018-01-01'),
                  end=pd.Timestamp('2018-01-04'),
                  freq='D')

### 创建`PeriodIndex`

使用`pd.period_range()`函数来创建`pd.PeriodIndex`对象：

In [ ]:
# 指定起始时间，周期数目，时间频率
pd.period_range(start='2018-01-01', periods=12, freq='W')

## 小结

在 Python 中的 `time`, `datetime`, `dateutil`, `numpy`, `pandads` 库都提供有与时间处理有关的对象与方法。还可以使用`AstroPy`等专业库来处理更复杂的天文时间。